In [ ]:
import pyarrow.dataset as ds
import polars as pl
from datetime import datetime
%load_ext memory_profiler
import time

In [ ]:
trade = ds.dataset("~/data/trade_taq2019.arrow", format = "arrow")
trade = pl.scan_ds(trade, allow_pyarrow_filter=True)

quote = ds.dataset("~/data/quote_taq2019.arrow", format = "arrow")
quote = pl.scan_ds(quote, allow_pyarrow_filter=True)

In [ ]:
trade.filter(
    (pl.col("Symbol") == "AAPL")&
    (pl.col("Sale_Condition").str.contains("O"))&
    (pl.col("Time").is_between(datetime(2019,10,7,9,30), datetime(2019,10,7,16,0)))
).collect()

In [ ]:
# sort the data first
trade = trade.sort("Time")

# query
trade.groupby_dynamic(index_column="Time", every="1h", period="1h", by="Symbol").agg(
[
    pl.col("Trade_Volume").sum().alias("Total_vol")
]).collect()

In [ ]:
# sort the data first
quote = quote.sort("Time")

# query
quote.groupby_dynamic(index_column="Time", every="1h", period="1h", by="Symbol").agg(
[
    (0.5*(pl.col("Bid_Price")+pl.col("Offer_Price")).mean()).alias("mid"),
        pl.col("Time").min().alias("min_time"),
        pl.col("Time").max().alias("max_time")
]).sort("Time").collect()